<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

Оценим загруженные данные...

In [3]:
print (df.info())
print ('\nКол-во дубликатов:',df.duplicated().sum())
print ('\nКол-во пропусков:\n',df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None

Кол-во дубликатов: 153

Кол-во пропусков:
 Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [4]:
df=df.astype('int')
df.drop_duplicates(inplace=True)
df=df.reset_index(drop=True)
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4842,0,28,35700,2,0
4843,0,34,52400,1,0
4844,0,20,33900,2,0
4845,1,22,32700,3,0


Преамбула....  
Немного освежим знания по элементарным действиям над матрицами, в частности  
$(AB)C=A(BC)$  
$(AB)^T=B^TA^T$  
$(AB)^{−1}=B^{−1}A^{−1}$  
$(A^T)^{−1}=(A^{−1})^T$  
$A^{−1}(AX)=(A^{−1}A)X=EX=X$

http://publish.sutd.ru/e_books/lin_alg_2013/html/matrix_33.html

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**  
Итак, чтобы ответить на это вопрос нужно сравнить рез-т предсказания "по дефолту" и предсказание при условии умножения на обратимую матрицу. Т.е. нужно сравнить значение $a = Xw$ и $a1=XRw1$, где $R(reversible)$-обратимая матрица  

Подставим в вектор весов Линейной Регрессии $w = (X^T X)^{-1} X^T y$ вместо $Х$ новое значение $XR$, получим:  
$w1 = ((XR)^T(XR))^{-1}(XR)^Ty$  
или  
$w1 = (X^TR^TXR)^{-1}(XR)^Ty$  
$w1 = (X^TXR^TR)^{-1}(XR)^Ty$ $-->$ $((X^TXR^TR)^{-1}$ представим как $(X^TX)^{-1}(R^T)^{-1}R^{-1}$, тогда  

$w1 = (X^TX)^{-1}(R^T)^{-1}R^{-1}(XR)^T y = (X^TX)^{-1}(R^T)^{-1}R^{-1}X^TR^T y$,  
$R$ является обратимой матрицей сл-но  $-->$ $(R^T)^{-1}(R^T) = E$  

тогда $w1 = (X^TX)^{-1}(R^T)^{-1}R^{-1}X^TR^T y = (X^TX)^{-1}ER^{-1}X^T y$ или $w1=(X^TX)^{-1}X^T yR^{-1}$  

$(X^TX)^{-1}X^T y$ это ничто иное как вектор весов линейной регрессии - $w$, т.е получаем $w1=wR^{-1}$  


Предсказание $a1$ выражается формулой $a1=XRw1$, где $w1=wR^{-1}$, таким образом, подставив $w1$,получаем:  

 <p style="text-align: center"> $a1=XRw1=XRR^{-1}w=XEw=Xw$, а это формула первичного предсказания, т.е.</p>  
 <p style="text-align: center"> $a1=Xw=a$</p>  
  
Таким образом мы видим, что предсказания "по умолчанию" и предсказания с применением обратимой матрицы равны друг другу, а это значит, что при умножении Признаков на обратимую матрицу, качество линейной регрессии не изменится.

## Алгоритм преобразования

**Алгоритм**

Учитывая выше изложенные свойства - отсутствия влияния обратимой матрицы на качество линейной регрессии (при умножении на Признаки), мы можем применить данную особенность в дальнейшем исследовании, а именно использовать в сокрытии-шифровании персональных данных.  
Поясню:
Представив Dataframe содержащий перс.данный пользователей в виде массива - матрицы, где столбцы являются Признаками и умножив эту матрицу на обратную матрицу (созданную случайным образом), мы превратим содержание Dataframe в бесполезный на первый вгляд набор данных, тем самым надежно скрыв персональную информацию пользователей.  
Итак, алгоритм следующий:  
1. Создаем случайную матрицу (ключ шифр)
2. Проверяем ее на обратимость (это важно, иначе если матрица не обратимая, то для нее нельзя найти обратную матрицу)  
(используем функцию numpy.linalg.inv() она также проверяет матрицу на обратимость, если матрица необратима, будет обнаружена ошибка)
3. Перемножаем Признаки на обратимую матрицу - Шифруем данные
4. Любуемся результатом
5. Расчитываем и сравниваем качество линейной регрессии до и после преобразования (на основе метрики R2)
6. Делаем выводы

## Проверка алгоритма

Могу предположить, что целевым признаком является столбец о наличие или отсутвие "страховой выплаты", тогда:

In [7]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [8]:
matrix = np.random.normal(size = (4,4)) # матрица должна быть квадратной, "ширина" определяется кол-вом Признаков.

In [9]:
matrix_inv = np.linalg.inv(matrix)

Проверяем как работает процесс шифрования/дешифрования.

In [10]:
# Исходные данные -  подлежащие шифрованию
features.values

array([[    1,    41, 49600,     1],
       [    0,    46, 38000,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33900,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40600,     1]])

In [11]:
# Шифрованные данные (ключем шифрования выступает matrix_inv)
new_features = features.values @ matrix_inv
new_features

array([[-44359.86255329,   9597.09469732, 243684.69883409,
        114480.36264421],
       [-33985.50716502,   7354.01933569, 186713.37633047,
         87726.18541755],
       [-18781.22264814,   4064.60931961, 103189.00046146,
         48485.21903089],
       ...,
       [-30318.85691846,   6557.69168003, 166537.64112228,
         78231.82086209],
       [-29246.67167645,   6326.28756662, 160647.5839908 ,
         75467.3459652 ],
       [-36310.75568415,   7855.04618347, 199460.02798238,
         93700.19034574]])

Отлично! Просто набор чисел... 

In [12]:
# Проверим дешифрование (при необходимости)
new_features = new_features @ matrix
new_features.astype('int')

array([[    0,    40, 49600,     1],
       [    0,    45, 38000,     1],
       [    0,    28, 21000,     0],
       ...,
       [    0,    19, 33900,     2],
       [    1,    21, 32700,     3],
       [    1,    27, 40600,     1]])

Прекрастно, при желании, все можно будет восстановить.  

Теперь перейдем к сравнению качества линейной регрессии до и после преобразования.

In [13]:
model = LinearRegression()

model.fit(features, target)
predictions = model.predict(features)

model.fit(new_features, target)
predictions_new = model.predict(new_features)

r2 = r2_score(target, predictions)
r2_new = r2_score(target, predictions_new)

print('Метрика R2 на исходных признаках:', r2)
print('Метрика R2 на зашифрованных признаках:', r2_new)
print('Дельта между исходной и зашифрованной метрикой равна', (r2_new-r2).astype('int'))

Метрика R2 на исходных признаках: 0.4302010046633359
Метрика R2 на зашифрованных признаках: 0.4302010046633302
Дельта между исходной и зашифрованной метрикой равна 0


Кстати обобщающая способность модели (LinearRegression()) так себе, метрика $R^2$  менее 0.5 (чем ближе к 1, тем лучше).

**Общий вывод**  
Качество прогнозироваия линейной регрессии до и после преобразовани признаков остается неизменным, это позволяет использовать данный метод шифрования в практических задачах. Я конечно не искушен в методах шифрования, но на мой взгляд, этот метод из категории "дешево и сердито".